## Exercise: Handling Exceptions
With the knowledge of tracebacks and how exceptions are raised and handled, it is time to practice some of these concepts using this notebook. We'll use a similar scenario where a program must read a configuration file called _config.txt_ and deal with errors as they show up.

In real-world scenarios, exceptions are more clearly deal with separate functions, so this exercise will have different functions for different tasks regarding the configuration file.

In [1]:
# Create a function that opens and reads the contents of the configuration file first
# Allow the function to accept a path. This will become handy later
def read_config(path):
    with open(path) as config_path:
        return config_path.readlines()

In [22]:
# To mimic a brand-new environment, define a temporary directory that will act as the normal destination for the configuration file
# In this way, we prevent polluting the current notebook environment
import tempfile
import os 

# Create the temporary directory
config_dir = tempfile.mkdtemp()
config_path = os.path.join(config_dir, "config.txt")
print(config_dir)
print(config_path)

/var/folders/nk/r3v6ztqj761035fb_f59wd8m0000gn/T/tmp6czyaczv
/var/folders/nk/r3v6ztqj761035fb_f59wd8m0000gn/T/tmp6czyaczv/config.txt


In [23]:
# Now try reading the newly defined config.txt path
read_config(config_path)

FileNotFoundError: [Errno 2] No such file or directory: '/var/folders/nk/r3v6ztqj761035fb_f59wd8m0000gn/T/tmp6czyaczv/config.txt'

A `FileNotFoundError`! In this case, we want to ensure that when there is no configuration file, a default configuration file is created with some basic values. Before catching exceptions, create another function that writes the default values for the configuration file

In [24]:
import uuid

default_config_contents = f"""# Rocket Ship Configuration File!
fuel_tanks = 4
oxygen_tanks = 3
initial_propulsing_level = 84
unique_key = {uuid.uuid4()}"""

def write_default_config(path):
    with open(path, 'w') as default_config:
        default_config.write(default_config_contents)

Now you have a `read_config()` function that will get the contents of the config file, a `write_default_config()` function that will write the default contents to a config file, so now you are ready to define the `main()` function that puts everything together. 

Write it up so that you can catch and handle the `FileNotFoundError` from before: 

In [16]:
def main():
    try:
        config_contents = read_config(config_path)
    except FileNotFoundError:
        write_default_config(config_path)
        config_contents = read_config(config_path)
    # Output the contents to verify all worked fine
    print(config_contents)
    
main()
    

['fuel_tanks = 4\n', 'oxygen_tanks = 3\n', 'initial_propulsing_level = 84\n', 'unique_key = 5a468ff2-96b1-4218-ba26-0400c49a5142']


The `main()` function now handles the `FileNotFoundError` and creates one by default with some contents. But so far all we have is a list with contents which is not that useful. Add a parser function that reads the values and creates a Python dictionary. Make the `main()` function output the dictionary after the parsing.

In [19]:
def parse_config(contents):
    config = {}
    for line in contents:
        key, value = line.split('=')
        config[key] = value
    return config

In [25]:
# Update the main() function to use the new parser
def main():
    try:
        config_contents = read_config(config_path)
    except FileNotFoundError:
        write_default_config(config_path)
        config_contents = read_config(config_path)
    # Output the contents to verify all worked fine
    print(parse_config(config_contents))
    
main()

ValueError: not enough values to unpack (expected 2, got 1)

Oh no, a `ValueError`. This is happening in the parser function. We now know we can handle this exception, but where? There are several functions involved and `main()` is currently handling exceptions. Where do you think this handling should happen?

Where to handle exceptions can be non-straightforward. There is no particular rule in Python that helps here - you can handle an exception anywhere in code. You must use common sense and see what fits best. Remember that inadequate handling can always be improved later, as everything in code. 

In this particular case, handling the specifics of parsing a configuration file should probably be the responsibility of the parser itself, so that it can continue parsing the entire file while handling errors within the parsing.

In [27]:
# update the parser to handle the ValueError
def parse_config(contents):
    config = {}
    for line in contents:
        try:
            key, value = line.split('=')
        except ValueError:
            # completely ignore this error and skip to the next line
            continue
        config[key] = value
    return config
    
# now call main() again and see if all is good now
main()

{'fuel_tanks ': ' 4\n', 'oxygen_tanks ': ' 3\n', 'initial_propulsing_level ': ' 84\n', 'unique_key ': ' 1f14d837-0bbb-4bcb-a2a2-7093ced5b236'}


Almost there. Somehow, the keys have trailing spaces, and some values have newline characters in them. Clean those up in the `parse_config()` function

In [34]:
def parse_config(contents):
    config = {}
    for line in contents:
        try:
            key, value = line.split('=')
        except ValueError:
            # completely ignore this error and skip to the next line
            continue
        config[key.strip()] = value.strip('\n').strip()
    return config

main()

{'fuel_tanks': '4', 'oxygen_tanks': '3', 'initial_propulsing_level': '84', 'unique_key': '1f14d837-0bbb-4bcb-a2a2-7093ced5b236'}


Good job! Now you have a few functions, each handling exceptions in their own way. Some functions delegate the error handling when doing the work (like the parsing one) and others like the `main()` function handle the logic at a higher level.